# Import Packages

In [1]:

import pandas as pd  # Importing pandas library as pd
import numpy as np  # Importing numpy library as np
import gspread  # Importing gspread library for Google Sheets
import panel as pn  # Importing panel library as pn
pn.extension('tabulator')  # Loading the tabulator extension in Panel
import hvplot.pandas  # Importing hvplot.pandas module for high-level plotting
import holoviews as hv  # Importing holoviews library as hv for data analysis and visualization
hv.extension('bokeh')  # Loading the bokeh extension in HoloViews for interactive plotting



# Connect your Bank Data and clean up your transactions

In [2]:
gc =gspread.service_account(filename='services-account.json')
sh = gc.open('BankStatement')

In [3]:
ws =sh.worksheet('Sheet1')
df=pd.DataFrame(ws.get_all_records())
df.head()

,Date,Transaction Description,Debit Amount,Credit Amount,Balance,Location
0,2023-01-01,Salary,0,3613,3613,
1,2023-01-02,Expense,322,0,3291,Marriott
2,2023-01-03,Salary,0,2634,5925,
3,2023-01-04,Expense,953,0,4972,Uber
4,2023-01-05,Salary,0,2796,7768,


In [4]:
df = df[['Date','Transaction Description','Debit Amount','Location']]
df['Transaction Description'] = df['Transaction Description'].map(str.lower)
df['Category'] ='unassigned'
df.head()

,Date,Transaction Description,Debit Amount,Location,Category
0,2023-01-01,salary,0,,unassigned
1,2023-01-02,expense,322,Marriott,unassigned
2,2023-01-03,salary,0,,unassigned
3,2023-01-04,expense,953,Uber,unassigned
4,2023-01-05,salary,0,,unassigned


In [5]:

# Food

df['Category'] = np.where(df['Location'].str.contains(
    'Starbucks|McDonalds'), 
    'Food', df['Category'] )
    
# Shopping
    
df['Category'] = np.where(df['Location'].str.contains(
    'Walmar|Targe|Amazon'), 
    'Shopping', df['Category'] )
    


# Cab Rental
    
df['Category'] = np.where(df['Location'].str.contains(
    'Uber|Lyft'), 
    'Cab Rental', df['Category'] )
    
# Travel
    
df['Category'] = np.where(df['Location'].str.contains(
    'American Airlines|Delta'), 
    'Travel', df['Category'] )
    
# Hotels
    
df['Category'] = np.where(df['Location'].str.contains(
    'Marriott|Hilton|Airbnb'), 
    'Hotels', df['Category'] )

#Salary
df['Category'] = np.where(df['Debit Amount'] == 0, 'Salary', df['Category'])


# Convert the "Date" column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
    
pd.options.display.max_rows = 999
df.head(200)

,Date,Transaction Description,Debit Amount,Location,Category,Month,Year
0,2023-01-01,salary,0,,Salary,1,2023
1,2023-01-02,expense,322,Marriott,Hotels,1,2023
2,2023-01-03,salary,0,,Salary,1,2023
3,2023-01-04,expense,953,Uber,Cab Rental,1,2023
4,2023-01-05,salary,0,,Salary,1,2023
5,2023-01-06,expense,153,Walmart,Shopping,1,2023
6,2023-01-07,salary,0,,Salary,1,2023
7,2023-01-08,expense,871,Marriott,Hotels,1,2023
8,2023-01-09,salary,0,,Salary,1,2023
9,2023-01-10,expense,680,Marriott,Hotels,1,2023


In [6]:
#check unassigned transactions and confirm all transactions are assigned to a category

unassigned = df.loc[df['Category'] == 'unassigned']
unassigned


,Date,Transaction Description,Debit Amount,Location,Category,Month,Year


## Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses and savings


In [7]:


# Get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

# Filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]


In [8]:
last_month_expenses = last_month_expenses.groupby('Category')['Debit Amount'].sum().reset_index()

last_month_expenses['Debit Amount']=last_month_expenses['Debit Amount'].astype('str')
last_month_expenses['Debit Amount ']=last_month_expenses['Debit Amount'].str.replace('-','')
last_month_expenses['Debit Amount']=last_month_expenses['Debit Amount'].astype('float')        #get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Salary") == False]    #exclude "excluded" category
last_month_expenses = last_month_expenses.sort_values(by='Debit Amount', ascending=False)    #sort values
last_month_expenses['Debit Amount'] = last_month_expenses['Debit Amount'].round().astype(int)      #round values

last_month_expenses

,Category,Debit Amount,Debit Amount
0,Cab Rental,923,923
3,Shopping,767,767
4,Travel,657,657
1,Hotels,239,239


In [9]:
last_month_expenses_tot = last_month_expenses['Debit Amount'].sum()
last_month_expenses_tot

2586

In [10]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")


Watcher(inst=TextInput(name='Non-Recurring Expenses', value='2586'), cls=<class 'panel.widgets.input.TextInput'>, fn=<function calculate_difference at 0x0000020513D5E160>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

## Create last month expenses bar chart 


In [11]:



last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Debit Amount', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 1000))

last_month_expenses_chart

:Bars   [Category]   (Debit Amount)

In [12]:
df['Date'] = pd.to_datetime(df['Date'])            # convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')    # extract the month and year from the 'Date' column and create a new column 'Month-Year'
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Debit Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Debit Amount']=monthly_expenses_trend_by_cat['Debit Amount'].astype('str')
monthly_expenses_trend_by_cat['Debit Amount']=monthly_expenses_trend_by_cat['Debit Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['Debit Amount']=monthly_expenses_trend_by_cat['Debit Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Salary") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Debit Amount', ascending=False)
monthly_expenses_trend_by_cat['Debit Amount'] = monthly_expenses_trend_by_cat['Debit Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Debit Amount': 'Amount '})

monthly_expenses_trend_by_cat


,Month-Year,Category,Amount
2,2023-01,Hotels,3637
7,2023-02,Hotels,2904
11,2023-03,Cab Rental,2415
13,2023-03,Hotels,2148
15,2023-03,Shopping,1709
16,2023-03,Travel,1681
9,2023-02,Shopping,1632
6,2023-02,Cab Rental,1321
1,2023-01,Food,1285
12,2023-03,Food,1233


In [13]:
select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Hotels',
    'Shopping',
    'Cab Rental',
    'Food',
    'Travel'
])

select_category1


Select(options=['All', 'Hotels', ...], value='All')

In [14]:
# define plot function
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot

# define callback function
@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# create layout
monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart


# ## Create summary table


Row
    [0] Select(options=['All', 'Hotels', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)

In [15]:
df = df[['Date', 'Category', 'Transaction Description', 'Debit Amount']]
df['Debit Amount']=df['Debit Amount'].astype('str')
df['Debit Amount']=df['Debit Amount'].str.replace('-','')
df['Debit Amount']=df['Debit Amount'].astype('float')        #get absolute figures

df = df[df["Category"].str.contains("Salary") == False]    #exclude "excluded" category
df['Debit Amount'] = df['Debit Amount'].round().astype(int)      #round values
df

,Date,Category,Transaction Description,Debit Amount
1,2023-01-02,Hotels,expense,322
3,2023-01-04,Cab Rental,expense,953
5,2023-01-06,Shopping,expense,153
7,2023-01-08,Hotels,expense,871
9,2023-01-10,Hotels,expense,680
11,2023-01-12,Shopping,expense,426
13,2023-01-14,Hotels,expense,550
15,2023-01-16,Shopping,expense,500
17,2023-01-18,Cab Rental,expense,262
19,2023-01-20,Hotels,expense,296


In [16]:
# Define a function to filter the dataframe based on the selected category
def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
select_category1.param.watch(update_summary_table, 'value')

summary_table


# ## Create Final Dashboard

DataFrame(height=300, sizing_mode='fixed', value=         Date  ..., width=400)

In [17]:


template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *If you can't manage your money, making more won't help*"),
        pn.pane.PNG('vecteezy_pack-of-dollars-money-clipart-design-illustration_9303600_278.png', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()

Launching server at http://localhost:50110


In [18]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
